In [4]:
# converting torch model to onnx
import os
import torch
import torch.nn as nn
from models import *
from IPython.display import Markdown, display

device = "cuda" if torch.cuda.is_available() else "cpu"


In [5]:
# Parameters
ONNX_SAVE_DIR_PATH = "trained_models/onnx/"
MODEL_PATH = "trained_models/its_train_ffa_3_9.pk"
IMAGE_HEIGHT = 240 #crop size
IMAGE_WIDTH = 240 #crop size
IR_MODEL_DIR = "trained_models/IR/"

In [6]:
def make_model(MODEL_PATH):
    _MODEL_NAME = MODEL_PATH.split('/')[-1]

    _MODEL_DATASET = _MODEL_NAME.split('_')[0]
    _MODEL_MODE = _MODEL_NAME.split('_')[1]
    _MODEL_ARCH = _MODEL_NAME.split('_')[2]
    _MODEL_NO_GROUPS = _MODEL_NAME.split('_')[3]
    _MODEL_NO_BLOCKS = _MODEL_NAME.split('_')[4]

    if _MODEL_ARCH == 'ffa':
        return FFA(gps = _MODEL_NO_GROUPS,blocks = _MODEL_NO_BLOCKS)
    print('models other than FFA not yet implemented')
    return None

In [7]:
MODEL_PATH = "trained_models/its_train_ffa_3_9.pk"
# dehaze_net = make_model(MODEL_PATH) # dynamic initialization #!TODO
dehaze_net = FFA(gps =3,blocks=9) # Manually initialize FFA net
dehaze_net = dehaze_net.to(device)
checkpoint = torch.load(MODEL_PATH)
dehaze_net.load_state_dict(checkpoint["model"])

<All keys matched successfully>

In [8]:
dummy_input = torch.randn(1, 3, 240, 240).to(device)
model_name = MODEL_PATH.split('/')[-1].split('.')[0]
onnx_model_save_path = ONNX_SAVE_DIR_PATH+ '/' + model_name + '.onnx'

# torch to onnx
torch.onnx.export(
        dehaze_net,
        dummy_input,
        onnx_model_save_path,
        opset_version=11,
        do_constant_folding=False,
)

## Convert ONNX model to OpenVINO IR Format

In [9]:
# Construct the command for Model Optimizer
mo_command = f"""mo
                 --input_model "{onnx_model_save_path}"
                 --input_shape "[1,3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
                 --data_type FP16
                 --output_dir "{IR_MODEL_DIR}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

Model Optimizer command to convert the ONNX model to OpenVINO:


`mo --input_model "trained_models/onnx//its_train_ffa_3_9.pth" --input_shape "[1,3, 240, 240]" --data_type FP16 --output_dir "trained_models/IR/"`

In [10]:
print("Exporting ONNX model to IR... This may take a few minutes.")
mo_result = %sx $mo_command
print("\n".join(mo_result))

Exporting ONNX model to IR... This may take a few minutes.
'mo' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import sys
import time
import os
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from openvino.inference_engine import IECore

In [ ]:
MODEL_DIR = "IR"
MODEL_NAME = "its_train_ffa_3_9"

os.makedirs(MODEL_DIR, exist_ok=True)

# Create filenames of the models that will be converted in this notebook.
model_path = Path(f"{MODEL_DIR}/{MODEL_NAME}")
ir_path = model_path.with_suffix(".xml")
onnx_path = model_path.with_suffix(".onnx")

In [ ]:
def resize_to_max_width(image, max_width):
    """
    Resize `image` to `max_width`, preserving the aspect ratio of the image.
    """
    if image.shape[1] > max_width:
        hw_ratio = image.shape[0] / image.shape[1]
        new_height = int(max_width * hw_ratio)
        image = cv2.resize(image, (max_width, new_height))
    return image

In [ ]:
# Load and prepare the IR model.
ie = IECore()
net = ie.read_network(ir_path)
exec_net = ie.load_network(net, "CPU")
input_key = next(iter(net.input_info.keys()))
output_key = next(iter(net.outputs.keys()))

In [ ]:
target_width = 224
target_height = 224
OUTPUT_DIR = "test"

In [ ]:
# Step 1. Load an image and convert to RGB
image_path = Path("data/RESIDE/val/1.jpg")
image = cv2.cvtColor(cv2.imread(str(image_path), flags=cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)

# Step 2. Transform the image (only resize and transpose are still required)
resized_image = cv2.resize(image, (target_width, target_height))
input_image = resized_image.transpose(2, 0, 1)[None, :, :, :]

# Step 3. Do inference.
result_ir = exec_net.infer({input_key: input_image})

# Step 4. Save the result image
anime_fn_ir = Path(f"{OUTPUT_DIR}/{image_path.stem}_anime_ir").with_suffix(".jpg")
if cv2.imwrite(str(anime_fn_ir), result_ir[:, :, (2, 1, 0)]):
    print(f"The anime image was saved to {anime_fn_ir}")